In [9]:
# Import the required libraries
import pandas as pd
import requests
import json
import time

# Get required API key related data'Priv
from config import appID, appKey, gkey
# Define base URL
url = 'https://maps.googleapis.com/maps/api/geocode/json'

In [10]:
# User params specific to all calls to this endpoint above
st = 'MN'
districtID = '2721240'
city = 'Minneapolis'
perPage = '50'
#school_levels = ['Elementary', 'Middle', 'High', 'Alt', 'Private']
school_levels = ['Elementary', 'Middle', 'High', 'Alt']

In [11]:
# Initialize series used to collect school data for distinct columns
schoolid = []
schoolName = []
latitude = []
longitude = []
latlng = []
year = []
numberOfStudents = []
percentFreeDiscLunch = []
pupilTeacherRatio = []
rankStars = []
rankLevel = []
rankStatewidePercentage = []
averageStandardScore = []
neighborhood = []

In [12]:
for level in school_levels:
    # Load JSON
    jsonFileName = f"./data/json_files/jsonFile{level}.json"
    with open(jsonFileName) as jsonfile:
        json_resp = json.load(jsonfile)
        #print(json.dumps(json_resp, indent=4, sort_keys=True))    
        # Create Json object to extract the relevant data for each school level
        schoolList = json_resp["schoolList"]
        # Figure out the number of schools in that particular school level category
        schoolNum = json_resp["numberOfSchools"]
        print(schoolNum)    
        for i in range (0, schoolNum):
            # Generic school info
            schoolid.append(schoolList[i]["schoolid"])        
            schoolName.append(schoolList[i]["schoolName"])
            latt = schoolList[i]["address"]["latLong"]["latitude"]
            latitude.append(schoolList[i]["address"]["latLong"]["latitude"])
            lont = schoolList[i]["address"]["latLong"]["longitude"]          
            longitude.append(schoolList[i]["address"]["latLong"]["longitude"])
            latlngt = f"{latt}, {lont}"
            latlng.append(latlngt)
            # School details - these fields were present in all datasets
            year.append(schoolList[i]["schoolYearlyDetails"][0]["year"])        
            numberOfStudents.append(schoolList[i]["schoolYearlyDetails"][0]["numberOfStudents"])
            percentFreeDiscLunch.append(schoolList[i]["schoolYearlyDetails"][0]["percentFreeDiscLunch"])
            pupilTeacherRatio.append(schoolList[i]["schoolYearlyDetails"][0]["pupilTeacherRatio"])
            # Rank History data was missing from various datasets - it needs to be handled separarely
            # Look for an empty section for a rankHistory section that is normally a dictionary
            if schoolList[i]["rankHistory"] is None:
                rankStars.append("NA")
                rankLevel.append("NA")
                rankStatewidePercentage.append("NA")
                averageStandardScore.append("NA")
            else:
                rankStars.append(schoolList[i]["rankHistory"][0]["rankStars"])
                rankLevel.append(schoolList[i]["rankHistory"][0]["rankLevel"])
                rankStatewidePercentage.append(schoolList[i]["rankHistory"][0]["rankStatewidePercentage"])
                averageStandardScore.append(schoolList[i]["rankHistory"][0]["averageStandardScore"])

43
10
42
7


In [5]:


    print(f"{schoolid} {schoolName} {latitude} {longitude} {latlng} {year} {rankStars} {rankLevel} {rankStatewidePercentage} \
    {averageStandardScore} {numberOfStudents} {percentFreeDiscLunch} {pupilTeacherRatio}")


['272124000932', '272124004172', '272124000938', '272124000941', '272124000942', '272124000943', '272124001878', '272124000945', '272124004778', '272124001868', '272124002582', '272124001870', '272124001882', '272124001866', '272124001879', '272124000970', '272124002580', '272124000978', '272124004239', '272124004742', '272124000986', '272124001014', '272124000989', '272124000991', '272124001861', '272124001057', '272124001920', '272124002476', '272124000999', '272124002144', '272124002699', '272124002177', '272124002296', '272124003434', '272124005048', '272124004991', '272124001031', '272124002178', '272124002297', '272124001054', '272124004885', '272124002700', '272124001865', '272124000936', '272124000937', '272124000965', '272124004882', '272124004574', '272124000969', '272124000988', '272124001008', '272124002581', '272124001028', '272124005036', '272124005000', '272124001922', '272124002862', '272124004930', '272124001896', '272124001903', '272124004937', '272124004940', '272124

In [ ]:
# User params specific to all calls to this endpoint above
for lat_lng in latlng:
    params = dict(latlng=lat_lng, key=gkey)
    res = requests.get(url, params=params)
    # Delay processing by five seconds due to API provider restrictions on usage
    time.sleep(5)
    json_resp = res.json()
    json_resp_tmp = json_resp['results'][0]['address_components'][2]['long_name']
    neighborhood.append(json_resp_tmp)

In [9]:
print(f"{schoolid} {schoolName} {latitude} {longitude} {latlng} {year} {rankStars} {rankLevel} {rankStatewidePercentage} \
    {averageStandardScore} {numberOfStudents} {percentFreeDiscLunch} {pupilTeacherRatio} {neighborhood}")

['272124000932', '272124004172', '272124000938', '272124000941', '272124000942', '272124000943', '272124001878', '272124000945', '272124004778', '272124001868', '272124002582', '272124001870', '272124001882', '272124001866', '272124001879', '272124000970', '272124002580', '272124000978', '272124004239', '272124004742', '272124000986', '272124001014', '272124000989', '272124000991', '272124001861', '272124001057', '272124001920', '272124002476', '272124000999', '272124002144', '272124002699', '272124002177', '272124002296', '272124003434', '272124005048', '272124004991', '272124001031', '272124002178', '272124002297', '272124001054', '272124004885', '272124002700', '272124001865', '272124000936', '272124000937', '272124000965', '272124004882', '272124004574', '272124000969', '272124000988', '272124001008', '272124002581', '272124001028', '272124005036', '272124005000', '272124001922', '272124002862', '272124004930', '272124001896', '272124001903', '272124004937', '272124004940', '272124

In [ ]:
neighborhood


In [ ]:
# Combine individual series above to create a dataframe consisting of all of them
minneapolis_schools = {'School ID': schoolid, 'School Name': schoolName, 'Latitude Longitude': latlng, 'Student Population': numberOfStudents,
                  'Free Discount Lunch': percentFreeDiscLunch, 'Students Per Teacher': pupilTeacherRatio, 'Rank Stars': rankStars, 
                  'Rank Level': rankLevel, 'Rank Statewide': rankStatewidePercentage, 'Average Standard Score': averageStandardScore,
                  'Neighborhood': neighborhood}
# Combine individual series above to create a dataframe consisting of all of them
minneapolis_schools = {'School ID': schoolid, 'School Name': schoolName, 'Latitude Longitude': latlng, 'Student Population': numberOfStudents,
                  'Free Discount Lunch': percentFreeDiscLunch, 'Students Per Teacher': pupilTeacherRatio, 'Rank Stars': rankStars, 
                  'Rank Level': rankLevel, 'Rank Statewide': rankStatewidePercentage, 'Average Standard Score': averageStandardScore,
                  'Neighborhood': neighborhood}

minneapolis_schools_df = pd.DataFrame(minneapolis_schools)  
minneapolis_schools_df


In [ ]:
# Output File (CSV)
output_data_file = "data/csv_files/minneapolis_schools.csv"
# Output data to an Excel CSV file
minneapolis_schools_df.to_csv(output_data_file)